# MERRA-2 Data Access

This code is designed to access reanalysis data from the Modern-Era Retrospective analysis for Research and Applications, Version 2 (MERRA-2). MERRA-2 is useful for its global data record of various land surface variables, including snow cover and snow depth.

In this example notebook, we are accessing the snow depth product, which is found in the "1-Hourly, Time-Averaged, Single-Level, Assimilation, Land Surface Diagnostics" product (M2T1NXLND), found here: https://disc.gsfc.nasa.gov/datasets/M2T1NXLND_5.12.4/summary.

In [1]:
import xarray as xr
import earthaccess
import boto3
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import warnings
from IPython.display import display, Markdown

Because MERRA-2 is a reanalysis product by NASA, we can get the data through `earthaccess`. In addition to using a `short_name` for the data product of interest, `earthaccess` also allows one to use the dataset's DOI for queries. The below query uses the DOI (`10.5067/RKPHT8KC1Y1T`) for M2T1NXLND.

In [2]:
# Authenticate using Earthdata Login prerequisite files
auth = earthaccess.login()

# Search for the granule by DOI
results = earthaccess.search_data(
    doi='10.5067/RKPHT8KC1Y1T',
    temporal=("2022-03-01", "2022-03-31"),
)

In [4]:
len(results)

31

The queried MERRA-2 data is organized such that 1 file = 1 day, so we should expect 31 files to be loaded below.

In [ ]:
# Access the MERRA-2 file(s) from the cloud
fn = earthaccess.open(results)

In [ ]:
# Open MERRA-2 data in Xarray (may be time-/memory-intensive if multiple files are queried)
ds = xr.open_mfdataset(fn)

With the above lines of code, we now have global land surface diagnostics for 744 time steps, or hourly over the month of March 2022.

Since we are interested in snow depth, we will focus on the `SNODP` variable, which provides snow depth in meters.

Let's look at global snow depths from March 1, 2022:

In [ ]:
# Sample global plot of snow depth at a single time
ds['SNODP'][0,:,:].plot(vmin=0, vmax=1)

We can also subset by latitude and longitude to look over a region of interest (Alaska, for this example).

In [ ]:
# Making bounds for Alaska
mask_lon = (ds.lon >= -168.75) & (ds.lon <= -136.01)
mask_lat = (ds.lat >= 52.64) & (ds.lat <= 71.59)

# Subset MERRA-2 data to Alaska lats/lons only
ds_ak = ds.where(mask_lon & mask_lat, drop=True)

In [ ]:
# Sample Alaska plot of snow depth at a single time
ds_ak['SNODP'][0,:,:].plot(vmin=0, vmax=1.25)

Finally, let's generate a time series of snow depth for the month of March 2022 near Fairbanks,

In [ ]:
# Resample snow depths to daily means
ak_daily_mean = ds_ak.resample(time='D').mean()

In [ ]:
# Plot daily mean snow depth over a month near Fairbanks, AK
ak_daily_mean['SNODP'][:,23,34].plot() 